## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [26]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Russell,US,32.3502,-85.1999,85.98,65,75,3.47,broken clouds
1,1,Vaini,TO,-21.2000,-175.2000,74.30,56,100,13.40,overcast clouds
2,2,Petropavlovsk-Kamchatskiy,RU,53.0452,158.6483,39.79,100,90,17.54,light rain
3,3,Busselton,AU,-33.6500,115.3333,56.08,76,12,10.85,few clouds
4,4,Hilo,US,19.7297,-155.0900,71.80,73,75,3.44,broken clouds


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp= float(input("What is the minimum temperature you would like for your trip?"))
max_temp= float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?80


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df= city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                     (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Rikitea,PF,-23.1203,-134.9692,75.33,80,100,10.11,overcast clouds
9,9,Sao Joao Da Barra,BR,-21.6403,-41.0511,78.89,74,12,5.61,few clouds
10,10,Rio Casca,BR,-20.2261,-42.6508,79.74,50,19,4.63,few clouds
22,22,Gat,IL,31.6100,34.7642,76.82,34,0,11.70,clear sky
55,55,Kapaa,US,22.0752,-159.3190,75.52,76,1,13.60,clear sky
56,56,Puerto Carreno,CO,6.1890,-67.4859,75.04,94,100,5.17,overcast clouds
57,57,Casablanca,MA,33.5928,-7.6192,77.22,64,0,17.00,clear sky
63,63,Atuona,PF,-9.8000,-139.0333,78.33,76,91,13.00,overcast clouds
71,71,Maragogi,BR,-9.0122,-35.2225,79.84,80,11,12.97,few clouds
74,74,San Policarpo,PH,12.1791,125.5072,79.12,86,99,4.16,moderate rain


In [29]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                102
City                   102
Country                101
Lat                    102
Lng                    102
Max Temp               102
Humidity               102
Cloudiness             102
Wind Speed             102
Current Description    102
dtype: int64

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()
clean_df.count()

City_ID                101
City                   101
Country                101
Lat                    101
Lng                    101
Max Temp               101
Humidity               101
Cloudiness             101
Wind Speed             101
Current Description    101
dtype: int64

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Rikitea,PF,75.33,overcast clouds,-23.1203,-134.9692,
9,Sao Joao Da Barra,BR,78.89,few clouds,-21.6403,-41.0511,
10,Rio Casca,BR,79.74,few clouds,-20.2261,-42.6508,
22,Gat,IL,76.82,clear sky,31.6100,34.7642,
55,Kapaa,US,75.52,clear sky,22.0752,-159.3190,
56,Puerto Carreno,CO,75.04,overcast clouds,6.1890,-67.4859,
57,Casablanca,MA,77.22,clear sky,33.5928,-7.6192,
63,Atuona,PF,78.33,overcast clouds,-9.8000,-139.0333,
71,Maragogi,BR,79.84,few clouds,-9.0122,-35.2225,
74,San Policarpo,PH,79.12,moderate rain,12.1791,125.5072,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat= row["Lat"]
    lng= row["Lng"]
    params["location"]= f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels= requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"]= hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")   

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [33]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df= hotel_df.dropna()

In [34]:
# 8a. Create the output File (CSV)
output_data_file= "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
max_temp=clean_hotel_df["Max Temp"]
fig= gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer= gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                               max_intensity=300, point_radius=4)
marker_layer= gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))